---
# Análise Fundamentalista
---
### Analisando um ativo da B3

In [ ]:
# instalação de bibliotecas
!pip install python-bcb
!pip install investpy
!pip install plotly
!pip install yfinance

In [2]:
# importação de bibliotecas
from bcb import sgs
import investpy
import plotly.graph_objects as go
import plotly.subplots
import pandas as pd
import io
import yfinance as yf
from plotly.subplots import make_subplots

**Coleta de dados macroeconômicos**

---

Precisamos analisar o comportamento dos indicadores macroeconomicos para avaliar os resultados da empresa.


In [3]:
#SELIC - Interest rate, Selic accumulated in the month in annual terms (basis 252)
selic = sgs.get({'selic':4189}, start= '2014-01-01')

A maioria das dívidas das empresas brasileiras são atreladas à taxa de juros, ou seja, quando essa taxa se eleva, a divida acaba sendo elevada em conjunto e prejudica os resultados demonstrados.

In [4]:
selic.head()

,selic
Date,
2014-01-01,10.17
2014-02-01,10.43
2014-03-01,10.65
2014-04-01,10.87
2014-05-01,10.90


In [5]:
# Gráfico SELIC
fig = go.Figure()
fig.add_trace(go.Scatter(x=selic.index, y=selic['selic']))
fig.update_layout(title_text= 'Selic Anual')
fig.show()

In [6]:
#IPCA - Índice nacional de preços ao consumidor-amplo (IPCA) em 12 meses
ipca = sgs.get({'ipca':13522}, start= '2014-01-01')

Indicador do "valor do dinheiro" do consumidor.

In [7]:
ipca.head()

,ipca
Date,
2014-01-01,5.59
2014-02-01,5.68
2014-03-01,6.15
2014-04-01,6.28
2014-05-01,6.37


In [8]:
# Gráfico IPCA
fig = go.Figure()
fig.add_trace(go.Scatter(x=ipca.index, y=ipca['ipca']))
fig.update_layout(title_text= 'IPCA 12 meses')
fig.show()

In [9]:
#CESTA BASICA SP - Cesta básica São Paulo
cesta_basica = sgs.get({'cesta_basica_sp':7493}, start= '2014-01-01')

Classes público alvo do Magazine Luiza consomem bastante os itens dispostos na cesta básica, então se esse custo se eleva, a renda disponível para comprar no varejo é reduzida.

In [10]:
cesta_basica.head()

,cesta_basica_sp
Date,
2014-01-01,323.47
2014-02-01,325.35
2014-03-01,351.46
2014-04-01,357.85
2014-05-01,366.54


In [11]:
# Gráfico Cesta Básica
fig = go.Figure()
fig.add_trace(go.Scatter(x=cesta_basica.index, y=cesta_basica['cesta_basica_sp']))
fig.update_layout(title_text= 'Cesta básica São Paulo')
fig.show()

**Coletando dados do ativo de interesse**

---

Neste exemplo, usaremos as Ações da empresa Magazine Luiza.

In [12]:
mglu = yf.download('MGLU3.SA', '2014-01-01', '2024-06-21', progress=False)

In [13]:
mglu.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,2.224315,2.285696,2.031405,2.113246,1.948873,4372391
2014-01-03,2.133706,2.189241,2.066480,2.183395,2.013566,2746258
2014-01-06,2.192164,2.309079,2.189241,2.297387,2.118690,2051739
2014-01-07,2.309079,2.417226,2.309079,2.355845,2.172602,4289596
2014-01-08,2.349999,2.408457,2.341231,2.402611,2.215730,2706229


In [14]:
# Gráfico MagaLu
fig = go.Figure()
fig.add_trace(go.Scatter(x=mglu.index, y=mglu['Adj Close']))
fig.update_layout(title_text= 'Preço MagaLu')
fig.show()

In [15]:
# Comportamento preços vs IPCA
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=ipca.index, y=ipca['ipca'], name='IPCA'), secondary_y=False)
fig.add_trace(go.Scatter(x=mglu.index, y=mglu['Adj Close'], name='MGLU-Preço'),secondary_y=True)
fig.update_layout(title_text='Preço vs IPCA')
fig.show()

Gráfico para acompanhar o impacto da variação do IPCA sobre o preço de Magazine Luiza.

In [16]:
#Verificando nível de incerteza no mercado observando a janela de voltilidade de 30 dias
mglu_retornos = mglu['Adj Close'].pct_change()  # cálculo de retorno diário
rol_vol = mglu_retornos.rolling(30).std()       # cálculo de desvio padrão em janela móvel(rolling) de 30 dias

fig = go.Figure()
fig.add_trace(go.Scatter(x=rol_vol.index, y=rol_vol))
fig.update_layout(title_text= 'Janela Volatilidade 30 dias')
fig.show()

Gráfico de volatilidade para entender se há consenso na decisão sobre o preço do papel. Uma alta volatilidade indicaria indecisão ao preço atual.

**Importando resultados MagaLu**

---

Dados fornecidos pelo RI da empresa, em seu site.

In [17]:
# fazendo upload do arquivo .xlsx
from google.colab import files
uploaded = files.upload()

Saving RESULTADO_1T24_POR.xlsx to RESULTADO_1T24_POR.xlsx


In [47]:
# "lendo" e transformando o arquivo .xlsx em um dataframe, indicando que a primeira linha será o cabeçalho
indicadores = pd.read_excel(io.BytesIO(uploaded['RESULTADO_1T24_POR.xlsx']), header=1)

In [48]:
# definindo o índice do dataframe
indicadores.set_index(indicadores['R$ milhões (exceto quando indicado)'],inplace=True)
# eliminando coluna obsoleta
indicadores.drop(columns=['R$ milhões (exceto quando indicado)'],inplace=True)
indicadores.head()

,1T18,2T18,3T18,4T18,1T19,2T19,3T19,4T19,1T20,2T20,...,3T23\nReapresentado,4T23,1T24,Unnamed: 26,2018,2019,2020,2021,2022,2023
R$ milhões (exceto quando indicado),,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vendas Totais (incluindo marketplace),4466.153,4618.764,4640.632,5942.298,5718.013,5746.969,6817.648,8988.062,7662.504,8566.476,...,14833.457,17947.425,16028.291,NaN,19667.847,27270.692,43516.670,55607.683,60160.731872,63056.273
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Receita Bruta Total,4366.290,4487.260,4444.450,5598.513,5313.213,5196.162,5999.434,7868.324,6486.285,6816.559,...,10571.016,13062.476,11530.064,NaN,18896.513,24377.133,36116.035,42982.687,45189.006000,45590.983
Receita Líquida Total,3613.263,3696.185,3670.467,4610.529,4328.984,4308.102,4864.198,6385.026,5234.749,5568.244,...,8578.818,10549.741,9239.265,NaN,15590.444,19886.310,29177.112,35278.150,37299.002000,36768.149


In [49]:
# eliminando outras colunas que não trazem informação relevante para a análise
indicadores.drop(columns=['Unnamed: 26', '2018', '2019', '2020', '2021','2022','2023'], axis=1, inplace=True)
indicadores.head()

,1T18,2T18,3T18,4T18,1T19,2T19,3T19,4T19,1T20,2T20,...,4T21,1T22,2T22,3T22,4T22,1T23,2T23,3T23\nReapresentado,4T23,1T24
R$ milhões (exceto quando indicado),,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vendas Totais (incluindo marketplace),4466.153,4618.764,4640.632,5942.298,5718.013,5746.969,6817.648,8988.062,7662.504,8566.476,...,15544.774,14124.251,13922.668872,14154.094,17959.718,15548.230,14727.161,14833.457,17947.425,16028.291
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Receita Bruta Total,4366.290,4487.260,4444.450,5598.513,5313.213,5196.162,5999.434,7868.324,6486.285,6816.559,...,11476.540,10576.940,10367.208000,10729.048,13515.810,11311.547,10645.944,10571.016,13062.476,11530.064
Receita Líquida Total,3613.263,3696.185,3670.467,4610.529,4328.984,4308.102,4864.198,6385.026,5234.749,5568.244,...,9399.996,8762.176,8562.389000,8807.019,11167.418,9067.334,8572.256,8578.818,10549.741,9239.265


**Analisando Receitas e Margens**

---

In [50]:
#Gráfico Ebitda vs Margem Ebitda
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=indicadores.columns, y=indicadores.loc['EBITDA'],name='EBITDA'), secondary_y=False)
fig.add_trace(go.Scatter(x=indicadores.columns, y=indicadores.loc['Margem EBITDA'],name='Margem EBITDA'), secondary_y=True)
fig.update_layout(title_text='EBITDA vs Margem EBITDA')
fig.show()

Gráfico para entender o comportamento do EBITDA relacionado à sua margem. São métricas para que seja possível fazer comparação de desempenho desta empresa com seus pares do mesmo setor.
*   **EBTIDA** (Earn before taxes interest depreciation and amortization) representa a receita operacional da empresa, geração de caixa operacional.
*   **Margem EBITDA** é o EBITDA dividido pela Receita Bruta.


In [51]:
# Gráfico Margem líquida vs Margem Ebtida
fig = go.Figure()
fig.add_trace(go.Scatter(x=indicadores.columns, y=indicadores.loc['Margem EBITDA'],name='Margem Ebtida'))
fig.add_trace(go.Scatter(x=indicadores.columns, y=indicadores.loc['Margem Líquida'], name='Margem Líquida'))
fig.update_layout(title_text='Margem Ebitda vs Margem Liquida')
fig.show()

Gráfico para entender o comportamento das margens de lucro. São métricas para que seja possível fazer comparação de desempenho desta empresa com seus pares do mesmo setor.
*   **Margem Líquida** é o Lucro Líquido dividido pela Receita Bruta.

**Canais de Venda**

---

Entender a receita em cada canal de venda e relacionar com as promessas estratégicas da diretoria da empresa

In [ ]:
canais = pd.read_excel(io.BytesIO(uploaded['RESULTADO_1T24_POR.xlsx']), header=1, sheet_name=16)
canais.set_index(canais['ABERTURA VENDAS TOTAIS'],inplace=True)
canais.drop(columns=['ABERTURA VENDAS TOTAIS'],inplace=True)

In [32]:
canais.head()

,1T18,2T18,3T18,4T18,1T19,2T19,3T19,4T19,1T20,2T20,...,3T23,4T23,1T24,Unnamed: 26,2018,2019,2020,2021,2022,2023
ABERTURA VENDAS TOTAIS,,,,,,,,,,,,,,,,,,,,,
Lojas virtuais,200.878746,216.018359,214.976,277.052,242.276,256.813,270.044,350.076,269.889,181.957,...,263.724,336.856,284.189,NaN,908.925105,1119.209000,1131.153,1187.480,1160.0910,1159.7250
Lojas convencionais,2687.766566,2876.294471,2744.255,3425.319,3108.175,3106.075,3251.884,4319.632,3306.103,1662.804,...,3712.879,4895.898,4290.712,NaN,11733.635038,13785.766000,13859.919,14668.698,15611.3703,16280.0108
Subtotal - Lojas Físicas,2888.645313,3092.31283,2959.231,3702.371,3350.451,3362.888,3521.928,4669.708,3575.992,1844.761,...,3976.603,5232.754,4574.901,NaN,12642.560143,14904.975000,14991.072,15856.178,16771.4613,17439.7358
E-commerce Tradicional (1P),1451.665317,1376.478785,1468.103,1873.934,1935.143,1801.274,2441.989173,3160.283,2854.962,4890.62,...,6444.199,7646.034,6812.513,NaN,6170.181102,9338.689173,20786.189,26688.664,27940.1220,27575.4150
Marketplace (3P),125.842689,149.973232,213.298,365.993,432.418,582.807,853.730829,1158.071,1231.545,1831.095,...,4412.657,5068.637,4640.877,NaN,855.106921,3027.026829,7739.403,13062.840,15449.1500,18041.1240


In [52]:
# Gráfico vendas por canal
fig = go.Figure()
fig.add_trace(go.Scatter(x=indicadores.columns, y=canais.loc['Subtotal - Lojas Físicas'],name='Lojas Físicas'))
fig.add_trace(go.Scatter(x=indicadores.columns, y=canais.loc['     E-commerce Tradicional (1P)'], name='1P'))
fig.add_trace(go.Scatter(x=indicadores.columns, y=canais.iloc[4], name='3P'))
fig.update_layout(title_text='Vendas por canal')
fig.show()

Gráficos de venda por canal, vemos que após a alta das vendas por e-commerce próprio (1P), esta se mantém em alta desde então, o comercio eletrônico por terceiros (3P) também se manteve em alta, equiparando-se com Lojas Físicas.

**Estrutura de Capital**

---

In [ ]:
estrutura_capital = pd.read_excel(io.BytesIO(uploaded['RESULTADO_1T24_POR.xlsx']),header=1, sheet_name=8)
estrutura_capital.set_index(estrutura_capital['ESTRUTURA DE CAPITAL (em R$ milhões)'],inplace=True)
estrutura_capital.drop(columns=['ESTRUTURA DE CAPITAL (em R$ milhões)'],inplace=True)

In [33]:
estrutura_capital.head()

,2018-03-31 00:00:00,2018-06-30 00:00:00,2018-09-30 00:00:00,2018-12-31 00:00:00,2019-03-31 00:00:00,2019-06-30 00:00:00,2019-09-30 00:00:00,2019-12-01 00:00:00,2020-03-31 00:00:00,2020-06-01 00:00:00,...,2023-09-05 00:00:00,2023-12-05 00:00:00,2024-03-01 00:00:00,Unnamed: 26,2018-12-31 00:00:00.1,2019-12-01 00:00:00.1,2020-12-01 00:00:00.1,2021-12-31 00:00:00.1,2022-12-04 00:00:00.1,2023-12-05 00:00:00.1
ESTRUTURA DE CAPITAL (em R$ milhões),,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(-) Empréstimos e Financiamentos Circulante,-381.416,-254.504,-252.410,-130.743,-128.911,-43.258,-313.387,-9.967,-6.477,-1650.78,...,-3002.748,-2954.347,-2269.425,NaN,-130.743,-9.967,-1667.181,-407.968,-124.297,-2954.347
(-) Empréstimos e Financiamentos não Circulante,-437.359,-327.383,-325.406,-325.224,-321.605,-1120.409,-832.697,-838.862,-847.372,-14.00,...,-4400.568,-4400.508,-4400.400,NaN,-325.224,-838.862,-19.581,-6384.904,-6984.460,-4400.508
(=) Endividamento Bruto,-818.775,-581.887,-577.816,-455.967,-450.516,-1163.667,-1146.084,-848.829,-853.849,-1664.78,...,-7403.316,-7354.855,-6669.825,NaN,-455.967,-848.829,-1686.762,-6792.872,-7108.757,-7354.855
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Gráfico de endividamento
fig = go.Figure()
fig.add_trace(go.Bar(x=indicadores.columns, y=estrutura_capital.loc['(-) Empréstimos e Financiamentos Circulante'],name='End. Curto Prazo'))
fig.add_trace(go.Bar(x=indicadores.columns, y=estrutura_capital.loc['(-) Empréstimos e Financiamentos não Circulante'],name='End. Longo Prazo'))
fig.update_layout(title_text='Perfil de Endividamento MGLU3')
fig.show()

Gráfico de endividamento, nos permite avaliar e analisar a dívida visualmente, de maneira nominal, se há alteração no perfil de endividamento ao longo do tempo.

In [54]:
# Gráfico Caixa Líquido Ajustado / EBITDA Ajustado
fig = go.Figure()
fig.add_trace(go.Scatter(x=indicadores.columns, y=estrutura_capital.loc['Caixa Líquido Ajustado / EBITDA Ajustado'],name='CaixaLiq/EBITDA'))
fig.update_layout(title_text='Caixa Liq. / EBITDA')
fig.show()

Gráfico para analisar caixa líquido ajustado sobre o EBITDA. Vai nos mostrar quantaz veze o EBITDA está cobrindo a dívida líquida e assim podemos avaliar a capacidade de operacional da empresa bancar suas dívidas.

*   **Caixa líquido** é o endividamento de curto prazo + endividamento de longo prazo - caixa - recebíveis.

**Estoques**

---


In [41]:
balanco_patr = pd.read_excel(io.BytesIO(uploaded['RESULTADO_1T24_POR.xlsx']), header=1, sheet_name=5)
balanco_patr.set_index(balanco_patr['ATIVO'],inplace=True)
balanco_patr.drop(columns=['ATIVO'],inplace=True)

In [45]:
balanco_patr.head()

,2018-03-31 00:00:00,2018-06-30 00:00:00,2018-09-30 00:00:00,2018-12-31 00:00:00,2019-03-31 00:00:00,2019-06-30 00:00:00,2019-09-30 00:00:00,2019-12-01 00:00:00,2020-03-31 00:00:00,2020-06-01 00:00:00,...,set/23\nReapresentado,2023-12-05 00:00:00,2024-03-01 00:00:00,Unnamed: 26,2018-12-31 00:00:00.1,2019-12-01 00:00:00.1,2020-12-01 00:00:00.1,2021-12-31 00:00:00.1,dez/22.1,2023-12-05 00:00:00.1
ATIVO,,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ATIVO CIRCULANTE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Caixa e Equivalentes de Caixa,775.152,680.451,419.013,599.087,293.189,625.705,221.794,305.746,388.904,1103.523,...,2804.023,2593.346,1978.265,NaN,599.087,305.746,1681.376,2566.218,2420.045,2593.346
Títulos e Valores Mobiliários,299.345,182.84,253.756,409.111,217.285,441.096,238.717,4448.158,2231.269,1878.803,...,480.829,779.072,352.061,NaN,409.111,4448.158,1221.779,1556.371,304.298,779.072
Contas a Receber - Cartão de Crédito,992.543,1018.947,1120.202,1492.316,1146.783,817.235,1141.985,2121.008,1365.742,3705.308,...,3618.397,4499.274,4698,NaN,1492.316,2121.008,3847.324,4618.014,5383.828,4499.274


In [55]:
# Gráfico de estoques
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=indicadores.columns, y=balanco_patr.loc['Estoques'],name='Estoques'), secondary_y=False)
fig.add_trace(go.Scatter(x=indicadores.columns, y=indicadores.loc['Receita Bruta Total'],name='Receita Bruta Total'), secondary_y=True)
fig.update_layout(title_text='Estoques')
fig.show()

Gráfico de estoques para entender se o nível de estoque justifica a provisão futura.